## ICESat-2 ATL03 SlideRule Demo

In [10]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sliderule import sliderule, icesat2, earthdata
import geopandas as gpd
from datetime import datetime
from datetime import timedelta
import json
import math

import warnings
from IPython import display
import json

warnings.filterwarnings('ignore')

In [11]:
url = "slideruleearth.io"
icesat2.init(url, verbose=False)
asset = "icesat2"

### Retrieve ATL03 elevations with several classification options

For two region of interest (blackrock, utqiagvik)

In [13]:
poly_fn

'../data/bbox_blackrock.geojson'

In [12]:
########## SET PARAMETERS ######################

###### Region of interest #########
site = "blackrock" # (blackrock, utqiagvik)
path = f"~/surfit/data/"

##### Read input parameters from .json files #####
poly_fn = f"../data/bbox_{site}.geojson"

pregion = gpd.read_file(poly_fn)
poly = sliderule.toregion(pregion)["poly"]

# Opening JSON file
f = open("../data/icesat2_tracks.json")
data = json.load(f)

granule_id = data[site]['granule_id']
track = int(data[site]['beam'][2])
pair = 0 if data[site]['beam'][3]=="l" else 1

time_start = datetime.strptime(granule_id[6:14], "%Y%m%d").strftime("%Y-%m-%d")
time_end = (datetime.strptime(granule_id[6:14], "%Y%m%d") + timedelta(days=1)).strftime("%Y-%m-%d")
rgt = int(granule_id[21:25])
cycle = int(granule_id[25:27])

filename = f"{path}{site}_ATL03_{data[site]['beam']}.csv"
print(filename)
########## SET PARAMETERS ######################

DriverError: ../data/bbox_blackrock.geojson: No such file or directory

## Calculate ATL06-SR Elevations from ATL03 Photons using SlideRule

In [ ]:
%%time

##### Set ATL03 sp parameters ##############################
# build sliderule parameters for ATL03 subsetting request
# SRT_LAND = 0
# SRT_OCEAN = 1
# SRT_SEA_ICE = 2
# SRT_LAND_ICE = 3
# SRT_INLAND_WATER = 4
parms = {
    # processing parameters
    "srt": 0,
    "len": 40,
    "res": 20,
    "track": track,
    # classification and checks
    # still return photon segments that fail checks
    "pass_invalid": True,
    # all photons
    "cnf": 0,
    "cnt": 5,
    "atl03_geo_fields": ["ref_azimuth", "ref_elev", "geoid"],
    # all land classification flags
    "atl08_class": ["atl08_noise", "atl08_ground", "atl08_canopy", "atl08_top_of_canopy", "atl08_unclassified"],
    # all photons
    "yapc": dict(knn=0, win_h=6, win_x=11, min_ph=4, score=0), 
}

# ICESat-2 data release
release = '006'

# find granule for each region of interest
granules_list = earthdata.cmr(short_name='ATL03', polygon=poly, time_start=time_start, time_end=time_end, version=release)

# create an empty geodataframe
parms["poly"] = poly
# gdf = icesat2.atl03sp(parms, asset=asset, version=release, resources=granules_list)
gdf = icesat2.atl03sp(parms, asset=asset, resources=granules_list)


In [ ]:
# Reduce dataframe for a single beam
def reduce_dataframe(gdf, RGT=None, GT=None, track=None, pair=None, cycle=None, beam='', crs=4326):
    # convert coordinate reference system
    D3 = gdf.to_crs(crs)
    # reduce to reference ground track
    if RGT is not None:
        D3 = D3[D3["rgt"] == RGT]
    # reduce to ground track (gt[123][lr]), track ([123]), or pair (l=0, r=1) 
    gtlookup = {icesat2.GT1L: 1, icesat2.GT1R: 1, icesat2.GT2L: 2, icesat2.GT2R: 2, icesat2.GT3L: 3, icesat2.GT3R: 3}
    pairlookup = {icesat2.GT1L: 0, icesat2.GT1R: 1, icesat2.GT2L: 0, icesat2.GT2R: 1, icesat2.GT3L: 0, icesat2.GT3R: 1}
    if GT is not None:
        D3 = D3[(D3["track"] == gtlookup[GT]) & (D3["pair"] == pairlookup[GT])]
    if track is not None:
        D3 = D3[D3["track"] == track]
    if pair is not None:
        D3 = D3[D3["pair"] == pair]
    # reduce to weak or strong beams
    # tested on cycle 11, where the strong beam in the pair matches the spacecraft orientation.
    # Need to check on other cycles
    if (beam == 'strong'):
        D3 = D3[D3['sc_orient'] == D3['pair']]
    elif (beam == 'weak'):
        D3 = D3[D3['sc_orient'] != D3['pair']]
    # reduce to cycle
    if cycle is not None:
        D3 = D3[D3["cycle"] == cycle]
    # otherwise, return both beams
    
    D3['x_atc'] = D3['segment_dist']+D3['distance']-np.min(D3['segment_dist'])

    # compute orthometric heights using the onboard geoid model (EGM08)
    D3['height_ortho'] = D3['height'] - D3['geoid']
    
    return D3

In [6]:
beam_type = 'strong'
project_srs = "EPSG:4326" #"EPSG:26912+EPSG:5703"
D3 = reduce_dataframe(gdf, RGT = rgt, track=track, pair = pair, beam=beam_type, crs=project_srs)

NameError: name 'reduce_dataframe' is not defined

In [7]:
print(len(D3))
D3.head()

NameError: name 'D3' is not defined

In [8]:
D3.keys()

NameError: name 'D3' is not defined

In [9]:
# Save geodataframe as csv
print(f'Saving file as {filename}')
D3.to_csv(filename)

NameError: name 'filename' is not defined